## 카테고리별 뉴스 제목, 기사, 언론사 크롤링 코드   

- 날짜 : 2022.09.01 ~ 2023.08.31   
- 카테고리 (예시) : <서울>   
- url 형식 : f"https://news.daum.net/breakingnews/society/nation/seoul?page={page}&regDate={date}"


In [ ]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import re
import requests
import pickle
from tqdm import tqdm

### 📅 날짜 생성

In [ ]:
# 날짜 생성
dates = pd.date_range("2023-01-01", "2023-01-31")

# 날짜에서 하이픈(-) 제거
kdates = [re.sub('-', '', str(date)[0:10]) for date in dates]
print(kdates)

### 📰 카테고리별 뉴스 크롤링 (전체 페이지)

- 주의사항) URL 안의 카테고리의 경우, 큰 카테고리도 있고 카테고리 안에 또 카테고리인 경우도 있어서 이 부분에 있어서는 직접 변경하는 것이 더 간편하다 생각하였음. 따라서 원하는 카테고리가 있다면 함수 내부의 url을 변경해야 함.

In [ ]:
def category_crawling(date):
    print('date =', date)

    all_news = []
    current_page = 1
    last = True

    while last == True:

        try:
            # 10 페이지씩 크롤링
            for page in tqdm(range(current_page, current_page + 10)):
                # print("page =", page)
                # 각 페이지에 접근
                url = f'https://news.daum.net/breakingnews/society/nation/seoul?page={page}&regDate={date}'
                res = requests.get(url)
                soup = bs(res.text, 'lxml')
                ul = soup.find("ul", {"class": "list_news2 list_allnews"}).findAll("li")


                for li in ul:
                    data = li.find("a", {"class": "link_txt"})
                    press = li.find("span", {"class": "info_news"}).text

                    news_url = data.get("href")
                    news_res = requests.get(news_url)
                    news_soup = bs(news_res.text, 'lxml')
                    article = news_soup.find("div", {"class": "article_view"}).find("section").findAll("p")[:-1]
                    contents = " ".join([p.text for p in article])

                    all_news.append({
                        'title': data.text,
                        'url': news_url,
                        'press': press,
                        'content': contents
                    })
        except Exception as e:
            print('오류내용 :', e)

        # "다음" 버튼이 있는지 확인
        try:
            next_button = soup.select_one('.btn_page.btn_next')

            # "다음" 버튼이 없으면 종료
            if not next_button:
                #print(f'페이지 {current_page}부터 {current_page + 9}까지 크롤링 완료')
                last = False
                # break
            else:
                # 다음 10 페이지로 이동
                current_page += 10

        except:
            print("페이지가 없음")

    return all_news

### 🏃 뉴스 크롤링

In [ ]:
seoul_crawling = [category_crawling(date) for date in kdates]

### ➡️ Dict -> DataFrame

In [ ]:
df_news = []
for news_list in seoul_crawling:
    df_news.extend(news_list)

news = pd.DataFrame(df_news)

final_news = news.drop_duplicates(subset='url')

print(f'중복제거된 기사 개수는 {len(news) - len(final_news)}입니다')

### 🥒 피클로 저장

In [ ]:
path = r'C:\0_Semi_project\data\seoul'

with open(path + '/seoul_20230101_20230131.pkl', mode='wb') as f:
    pickle.dump(final_news, f)